In [1]:
# Use GAN to generate synthesized characters on MNIST dataset

In [2]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image
from torchsummary import summary as PyTorchSummary

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [6]:
latent_size = 10
hidden_size1 = 256
hidden_size2 = 64
image_size = 28*28
num_epochs = 200
batch_size = 100
sample_dir = 'samples'

In [7]:
# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [8]:
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = (0.5,0.5,0.5),
                                     std = (0.5, 0.5, 0.5))])

In [9]:
# MNIST dataset

In [10]:
mnist = torchvision.datasets.MNIST(root='./../data',
                                          train = True,
                                          transform = transform,
                                          download = True)

In [11]:
data_loader = torch.utils.data.DataLoader(dataset = mnist,
                                          batch_size = batch_size,
                                          shuffle = True)

## Build model

In [12]:
# Discriminator: a simple classification model

In [15]:
D = nn.Sequential(
    nn.Linear(image_size, hidden_size1),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size1, hidden_size2),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size2, 1),
    nn.Sigmoid())

In [16]:
PyTorchSummary(D, input_size = (1,image_size))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 1, 256]         200,960
         LeakyReLU-2               [-1, 1, 256]               0
            Linear-3                [-1, 1, 64]          16,448
         LeakyReLU-4                [-1, 1, 64]               0
            Linear-5                 [-1, 1, 1]              65
           Sigmoid-6                 [-1, 1, 1]               0
Total params: 217,473
Trainable params: 217,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.83
Estimated Total Size (MB): 0.84
----------------------------------------------------------------


In [17]:
# Generator: like an autoencoder

In [18]:
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size2),
    nn.ReLU(),
    nn.Linear(hidden_size2, hidden_size1),
    nn.ReLU(),
    nn.Linear(hidden_size1, image_size),
    nn.Tanh())

In [19]:
PyTorchSummary(G, input_size = (1, latent_size))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 64]             704
              ReLU-2                [-1, 1, 64]               0
            Linear-3               [-1, 1, 256]          16,640
              ReLU-4               [-1, 1, 256]               0
            Linear-5               [-1, 1, 784]         201,488
              Tanh-6               [-1, 1, 784]               0
Total params: 218,832
Trainable params: 218,832
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 0.83
Estimated Total Size (MB): 0.85
----------------------------------------------------------------


In [20]:
# Device setting
D = D.to(device)
G = G.to(device)


In [21]:
# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0004)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.001)

In [22]:
def denorm(x):
    out = (x+1)/2
    return out.clamp(0, 1)

In [23]:
def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

## Training

In [24]:
total_step = len(data_loader)

In [ ]:
for epoch in range(num_epochs):
    for i, (images,_) in enumerate(data_loader):
        images = images.reshape(batch_size,-1).to(device)
        
        #create labels with are later used as input for BCE loss
        real_labels = torch.ones(batch_size,1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)
        
        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #
        
        #compute BCE_loss using real images where BCE_loss(x,y) = -y*log(D(x)) - (1-y)*log(1-D(x))
        #second term of the loss is always zero for real cases because y=1 for real case
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        #compute BCEloss using fake images
        #First term of the loss is always zero for fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        #backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        
        #train G to maximize log(D(G(z))) instead of minimizing log(1-D(G(z)))
        #use real label to keep the first term so loss = -log(D(G(z)))
        g_loss = criterion(outputs, real_labels)
        
        #back_prop
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
            
    # Save real images
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
        
    # Save sampled images
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))
    

Epoch [0/200], Step [200/600], d_loss: 0.2983, g_loss: 2.4733, D(x): 0.95, D(G(z)): 0.20
Epoch [0/200], Step [400/600], d_loss: 0.4694, g_loss: 4.9020, D(x): 0.82, D(G(z)): 0.09
Epoch [0/200], Step [600/600], d_loss: 1.4007, g_loss: 1.5156, D(x): 0.58, D(G(z)): 0.48
Epoch [1/200], Step [200/600], d_loss: 0.1328, g_loss: 3.4951, D(x): 0.94, D(G(z)): 0.07
Epoch [1/200], Step [400/600], d_loss: 0.9246, g_loss: 1.8409, D(x): 0.69, D(G(z)): 0.34
Epoch [1/200], Step [600/600], d_loss: 0.4921, g_loss: 2.1683, D(x): 0.82, D(G(z)): 0.21
Epoch [2/200], Step [200/600], d_loss: 1.5422, g_loss: 1.3270, D(x): 0.58, D(G(z)): 0.46
Epoch [2/200], Step [400/600], d_loss: 1.4908, g_loss: 0.8859, D(x): 0.50, D(G(z)): 0.48
Epoch [2/200], Step [600/600], d_loss: 0.2141, g_loss: 3.3353, D(x): 0.91, D(G(z)): 0.10
Epoch [3/200], Step [200/600], d_loss: 0.8342, g_loss: 3.1209, D(x): 0.73, D(G(z)): 0.24
Epoch [3/200], Step [400/600], d_loss: 0.1518, g_loss: 4.3182, D(x): 0.94, D(G(z)): 0.07
Epoch [3/200], Step [

Epoch [30/200], Step [400/600], d_loss: 0.5349, g_loss: 2.9304, D(x): 0.85, D(G(z)): 0.12
Epoch [30/200], Step [600/600], d_loss: 0.4370, g_loss: 4.7951, D(x): 0.84, D(G(z)): 0.05
Epoch [31/200], Step [200/600], d_loss: 0.5553, g_loss: 3.1213, D(x): 0.81, D(G(z)): 0.14
Epoch [31/200], Step [400/600], d_loss: 0.4408, g_loss: 3.4682, D(x): 0.88, D(G(z)): 0.17
Epoch [31/200], Step [600/600], d_loss: 0.7228, g_loss: 3.5245, D(x): 0.80, D(G(z)): 0.19
Epoch [32/200], Step [200/600], d_loss: 0.3889, g_loss: 3.5453, D(x): 0.86, D(G(z)): 0.10
Epoch [32/200], Step [400/600], d_loss: 0.4663, g_loss: 3.1515, D(x): 0.84, D(G(z)): 0.09
Epoch [32/200], Step [600/600], d_loss: 0.6792, g_loss: 2.6904, D(x): 0.85, D(G(z)): 0.16
Epoch [33/200], Step [200/600], d_loss: 0.3368, g_loss: 3.3530, D(x): 0.87, D(G(z)): 0.11
Epoch [33/200], Step [400/600], d_loss: 0.3510, g_loss: 3.5171, D(x): 0.88, D(G(z)): 0.07
Epoch [33/200], Step [600/600], d_loss: 0.4868, g_loss: 4.4965, D(x): 0.89, D(G(z)): 0.16
Epoch [34/

Epoch [60/200], Step [600/600], d_loss: 0.6143, g_loss: 2.9639, D(x): 0.79, D(G(z)): 0.18
Epoch [61/200], Step [200/600], d_loss: 0.6203, g_loss: 3.1708, D(x): 0.81, D(G(z)): 0.21
Epoch [61/200], Step [400/600], d_loss: 0.8233, g_loss: 2.4406, D(x): 0.70, D(G(z)): 0.19
Epoch [61/200], Step [600/600], d_loss: 0.6083, g_loss: 2.5222, D(x): 0.76, D(G(z)): 0.14
Epoch [62/200], Step [200/600], d_loss: 0.9495, g_loss: 2.4787, D(x): 0.67, D(G(z)): 0.20
Epoch [62/200], Step [400/600], d_loss: 0.6504, g_loss: 2.6829, D(x): 0.78, D(G(z)): 0.15
Epoch [62/200], Step [600/600], d_loss: 0.7440, g_loss: 2.1958, D(x): 0.76, D(G(z)): 0.24
Epoch [63/200], Step [200/600], d_loss: 0.7602, g_loss: 1.9564, D(x): 0.71, D(G(z)): 0.16
Epoch [63/200], Step [400/600], d_loss: 0.6717, g_loss: 2.5454, D(x): 0.87, D(G(z)): 0.30
Epoch [63/200], Step [600/600], d_loss: 0.6289, g_loss: 2.8016, D(x): 0.75, D(G(z)): 0.13
Epoch [64/200], Step [200/600], d_loss: 0.5594, g_loss: 2.0593, D(x): 0.79, D(G(z)): 0.18
Epoch [64/

Epoch [91/200], Step [200/600], d_loss: 0.7567, g_loss: 1.7339, D(x): 0.78, D(G(z)): 0.29
Epoch [91/200], Step [400/600], d_loss: 0.5865, g_loss: 1.9919, D(x): 0.77, D(G(z)): 0.18
Epoch [91/200], Step [600/600], d_loss: 0.7180, g_loss: 2.0829, D(x): 0.78, D(G(z)): 0.26
Epoch [92/200], Step [200/600], d_loss: 0.9335, g_loss: 1.8187, D(x): 0.70, D(G(z)): 0.25
Epoch [92/200], Step [400/600], d_loss: 0.9695, g_loss: 1.8997, D(x): 0.67, D(G(z)): 0.27
Epoch [92/200], Step [600/600], d_loss: 0.7397, g_loss: 2.0098, D(x): 0.71, D(G(z)): 0.17
Epoch [93/200], Step [200/600], d_loss: 0.6841, g_loss: 2.0172, D(x): 0.79, D(G(z)): 0.26
Epoch [93/200], Step [400/600], d_loss: 0.8861, g_loss: 1.7584, D(x): 0.68, D(G(z)): 0.24
Epoch [93/200], Step [600/600], d_loss: 0.9658, g_loss: 1.7132, D(x): 0.66, D(G(z)): 0.28
Epoch [94/200], Step [200/600], d_loss: 0.6730, g_loss: 1.7943, D(x): 0.72, D(G(z)): 0.18
Epoch [94/200], Step [400/600], d_loss: 0.7363, g_loss: 2.2802, D(x): 0.69, D(G(z)): 0.16
Epoch [94/

Epoch [121/200], Step [200/600], d_loss: 1.1189, g_loss: 1.8736, D(x): 0.62, D(G(z)): 0.29
Epoch [121/200], Step [400/600], d_loss: 0.8204, g_loss: 1.6865, D(x): 0.74, D(G(z)): 0.29
Epoch [121/200], Step [600/600], d_loss: 0.9308, g_loss: 1.5080, D(x): 0.80, D(G(z)): 0.40
Epoch [122/200], Step [200/600], d_loss: 0.8114, g_loss: 1.5340, D(x): 0.82, D(G(z)): 0.36
Epoch [122/200], Step [400/600], d_loss: 0.9470, g_loss: 1.4682, D(x): 0.64, D(G(z)): 0.28
Epoch [122/200], Step [600/600], d_loss: 0.8443, g_loss: 1.6368, D(x): 0.77, D(G(z)): 0.32
Epoch [123/200], Step [200/600], d_loss: 0.9694, g_loss: 1.5488, D(x): 0.66, D(G(z)): 0.27
Epoch [123/200], Step [400/600], d_loss: 0.8140, g_loss: 1.1010, D(x): 0.76, D(G(z)): 0.31
Epoch [123/200], Step [600/600], d_loss: 0.8824, g_loss: 1.6500, D(x): 0.71, D(G(z)): 0.29
Epoch [124/200], Step [200/600], d_loss: 0.9763, g_loss: 1.3629, D(x): 0.72, D(G(z)): 0.37
Epoch [124/200], Step [400/600], d_loss: 0.7768, g_loss: 1.5145, D(x): 0.80, D(G(z)): 0.32

Epoch [151/200], Step [200/600], d_loss: 1.0329, g_loss: 1.2813, D(x): 0.66, D(G(z)): 0.32
Epoch [151/200], Step [400/600], d_loss: 0.8405, g_loss: 1.5073, D(x): 0.71, D(G(z)): 0.30
Epoch [151/200], Step [600/600], d_loss: 1.0739, g_loss: 1.3861, D(x): 0.65, D(G(z)): 0.34
Epoch [152/200], Step [200/600], d_loss: 0.9835, g_loss: 1.6443, D(x): 0.73, D(G(z)): 0.37
Epoch [152/200], Step [400/600], d_loss: 0.8836, g_loss: 1.4956, D(x): 0.70, D(G(z)): 0.30
Epoch [152/200], Step [600/600], d_loss: 0.9728, g_loss: 1.3442, D(x): 0.65, D(G(z)): 0.31
Epoch [153/200], Step [200/600], d_loss: 1.0242, g_loss: 1.4783, D(x): 0.61, D(G(z)): 0.27
Epoch [153/200], Step [400/600], d_loss: 0.7706, g_loss: 1.3019, D(x): 0.75, D(G(z)): 0.30
Epoch [153/200], Step [600/600], d_loss: 1.0083, g_loss: 1.2884, D(x): 0.66, D(G(z)): 0.33
Epoch [154/200], Step [200/600], d_loss: 0.9180, g_loss: 1.6061, D(x): 0.73, D(G(z)): 0.35
Epoch [154/200], Step [400/600], d_loss: 1.1210, g_loss: 1.5485, D(x): 0.59, D(G(z)): 0.33